In [16]:
# =========================================================
# PalmOracle — Age & Fortune
# =========================================================
# !pip -q install gradio==4.* pillow numpy tensorflow

import os, random, json
import numpy as np
from PIL import Image
import gradio as gr
import tensorflow as tf

# ----------------- CONFIG (SỬA THEO BẠN) -----------------
MODEL_PATH  = "/content/hand_age_classifier.h5"             # <— đường dẫn model đã train
CLASS_NAMES = ["Adults", "Seniors", "Teenagers"]    # <— đúng thứ tự nhãn khi train
APP_NAME    = "PalmOracle — Age & Fortune"
TAU_ABSTAIN = 0.60   # dưới ngưỡng này sẽ báo "chưa chắc"
# ----------------------------------------------------------

# 1) Load model & lấy input shape (tự nhận 1/3 kênh)
model = tf.keras.models.load_model(MODEL_PATH)
input_h, input_w, input_c = model.input_shape[1:4]

# 2) Fortunes vui vẻ (giải trí)
FORTUNES = {
    "Teenagers": [
        "Bạn học nhanh gấp đôi khi biến tò mò thành thói quen mỗi ngày 15 phút.",
        "Một chiếc notebook nhỏ sẽ giữ được 3 ý tưởng lớn tuần này.",
        "Bạn sắp gặp một người bạn hợp gu — mở lời trước nhé!",
        "Lỗi sai hôm nay là bước đệm cho highlight của tháng tới."
    ],
    "Adults": [
        "Một mối quan hệ công việc lỏng lẻo hôm nay sẽ thành đồng minh chiến lược.",
        "Thêm 30 phút vận động/tuần sẽ kéo mood của bạn lên thấy rõ.",
        "Dự án sắp tới giúp bạn nâng cấp kỹ năng gấp đôi — cứ mạnh dạn nhận.",
        "Kỷ luật tài chính nhỏ mỗi ngày tạo khác biệt lớn trong 90 ngày."
    ],
    "Seniors": [
        "Kho kinh nghiệm của bạn là lợi thế cạnh tranh — kể lại câu chuyện của mình nhé.",
        "Một chuyến đi ngắn sẽ gỡ nút thắt bạn suy nghĩ bấy lâu.",
        "Ngủ tốt là khoản đầu tư sinh lời cao nhất cho sức khỏe.",
        "Bạn truyền cảm hứng tự nhiên — ai đó đang đợi điều đó từ bạn."
    ]
}

# ----------------- THEME -----------------
def _theme_vars(mode="Light"):
    if mode == "Dark":
        return f"""
        <style id='theme-vars'>
        :root {{
          --app-text:#e5e7eb; --app-muted:#cbd5e1; --app-border:#1f2937; --app-primary:#38bdf8;
          --header-start:#0f172a; --header-end:#0b2e4a; --header-shadow:rgba(56,189,248,.25);
          --slab-bg:#0b1220; --slab-border:#1f2937; --card-bg:#1f2937; --card-border:#334155; --card-text:#fef3c7;
          --page-bg:#0a0f1a;
          --title-font:"Outfit", system-ui, -apple-system, Segoe UI, Roboto, Arial;
          --text-font:"Inter", system-ui, -apple-system, Segoe UI, Roboto, Arial;
        }}
        body {{ background: var(--page-bg) !important; }}
        </style>
        """
    else:
        return f"""
        <style id='theme-vars'>
        :root {{
          --app-text:#0b1220; --app-muted:#374151; --app-border:#e5e7eb; --app-primary:#0ea5e9;
          --header-start:#e0f2fe; --header-end:#bae6fd; --header-shadow:rgba(2,132,199,.12);
          --slab-bg:#ffffff; --slab-border:#e5e7eb; --card-bg:#fff7ed; --card-border:#fdba74; --card-text:#7c2d12;
          --page-bg:#f8fafc;
          --title-font:"Outfit", system-ui, -apple-system, Segoe UI, Roboto, Arial;
          --text-font:"Inter", system-ui, -apple-system, Segoe UI, Roboto, Arial;
        }}
        body {{ background: var(--page-bg) !important; }}
        </style>
        """

BASE_CSS = """
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&family=Outfit:wght@700;800&display=swap');
.gradio-container { max-width: 1024px !important; }
* { letter-spacing:.1px; font-family: var(--text-font); color: var(--app-text); }
.header {
  background: linear-gradient(135deg, var(--header-start), var(--header-end));
  color: var(--app-text);
  padding: 22px 20px; border-radius: 18px; margin-bottom: 14px;
  border: 1px solid var(--app-border);
  box-shadow: 0 8px 24px var(--header-shadow);
}
.header h1 { font-family: var(--title-font); font-weight:800; font-size: 30px; margin:0 0 8px 0;}
.header p  { margin:0; font-size:15px; opacity:.95; line-height:1.5; }
.slab { background: var(--slab-bg); border:1px solid var(--slab-border);
        border-radius:16px; padding:12px; }
.card { background: var(--card-bg); border:1px solid var(--card-border);
        color: var(--card-text); border-radius:16px; padding:14px; font-weight:600; }
button, .gr-button { font-weight:700; }
.gr-button-primary { background: var(--app-primary) !important; color: #fff !important; }
.gr-image { border:1px solid var(--slab-border); border-radius: 10px; }
.big-title { font-family: var(--title-font); font-size: 26px; margin: 6px 0 10px 0;}
.footer { text-align:center; font-size:12px; opacity:.7; margin-top: 12px;}
"""

# ----------------- TIỀN XỬ LÝ ẢNH -----------------
def _to_rgb_or_gray(pil_img, want_c):
    return pil_img.convert("L") if want_c == 1 else pil_img.convert("RGB")

def preprocess(pil_img):
    img = _to_rgb_or_gray(pil_img, input_c)
    img = img.resize((input_w, input_h), Image.BILINEAR)
    arr = np.array(img, dtype=np.float32) / 255.0
    if input_c == 1:
        arr = arr[..., np.newaxis]
    arr = arr[np.newaxis, ...]      # (1,H,W,C)
    return arr

# ----------------- DỰ ĐOÁN -----------------
def _to_probs(vec):
    """Đảm bảo trả về xác suất hợp lệ (kể cả khi model xuất logits)."""
    v = np.asarray(vec, dtype=np.float32).ravel()
    if np.any(v < 0) or v.sum() <= 0.99 or v.sum() >= 1.01:
        # coi như logits → softmax
        e = np.exp(v - v.max())
        v = e / (e.sum() + 1e-8)
    return v

def predict(image, theme_choice):
    if image is None:
        return ("### —", {}, "Hãy tải một ảnh lòng bàn tay!", "—", None, _theme_vars(theme_choice))

    arr = preprocess(image)
    raw = model.predict(arr, verbose=0)[0]
    probs = _to_probs(raw)

    # Nếu số chiều không khớp CLASS_NAMES, cắt/ngắt cho khớp (phòng lỗi)
    k = min(len(probs), len(CLASS_NAMES))
    probs = probs[:k] / (probs[:k].sum() + 1e-8)

    idx = int(np.argmax(probs))
    pred_class = CLASS_NAMES[idx]
    conf = float(probs[idx])

    conf_dict = {CLASS_NAMES[i]: float(probs[i]) for i in range(k)}

    if conf < TAU_ABSTAIN:
        title_md = "## Chưa chắc"
        summary  = f"**Chưa chắc** — độ tin cậy ~ {conf*100:.1f}%. Hãy chụp gần, rõ lòng bàn tay, ánh sáng đều."
        fortune  = "Khi bạn kiên nhẫn thêm một chút, kết quả cũng kiên nhẫn đến với bạn."
    else:
        title_md = f"## {pred_class}"
        summary  = f"**Nhóm tuổi dự đoán:** {pred_class} — **độ tin cậy ~ {conf*100:.1f}%**"
        fortune  = random.choice(FORTUNES.get(pred_class, ["Chúc bạn một ngày rực rỡ!"]))

    return (title_md, conf_dict, summary, fortune, image, _theme_vars(theme_choice))

# ----------------- UI -----------------
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue"),
               css=BASE_CSS,
               title=APP_NAME) as demo:

    theme_vars = gr.HTML(_theme_vars("Light"))  # hidden theme vars

    gr.HTML(f"""
    <div class="header">
      <h1>🖐️ {APP_NAME}</h1>
      <p>Tải ảnh lòng bàn tay — hệ thống dự đoán <b>nhóm tuổi</b> trong 3 lứa tuổi
      và đưa ra một <b>“dự đoán vui vẻ”</b>.<br>
      <span>Chỉ phục vụ học tập & giải trí; không mang tính y khoa hay tư vấn đời sống.</span></p>
    </div>
    """)

    with gr.Row():
        with gr.Column(scale=1, min_width=360):
            inp = gr.Image(type="pil", sources=["upload","webcam"],
                           label="Upload / Webcam", image_mode="RGB", height=320)
            theme_toggle = gr.Radio(choices=["Light","Dark"], value="Light",
                                    label="Theme", interactive=True)
            btn = gr.Button("✨ Predict Now", variant="primary")

        with gr.Column(scale=1, elem_classes=["slab"]):
            big_title = gr.Markdown("### —", elem_classes=["big-title"])
            lbl = gr.Label(label="Prediction (probabilities)")
            txt = gr.Markdown(value="—")
            fortune_box = gr.Markdown(value="—", elem_classes=["card"])

    with gr.Row():
        with gr.Column(scale=1):
            out_img = gr.Image(label="Preview", height=260)

    gr.HTML('<div class="footer">© 2025 — Built for study & fun • No medical/biometric claims</div>')

    btn.click(fn=predict,
              inputs=[inp, theme_toggle],
              outputs=[big_title, lbl, txt, fortune_box, out_img, theme_vars])
    theme_toggle.change(lambda m: _theme_vars(m), inputs=theme_toggle, outputs=theme_vars)

demo.launch(share=True, show_error=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9043b1ab26c6430ba3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
